In [1]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [28]:
df = pd.read_csv("skscope_experiment_lasso.csv")

df[df["method"] == "LASSO"].groupby("model")[["accuracy", "time"]].agg(
    lambda x: str(x.mean().round(2)) + "(" + str(x.std().round(2)) + ")"
).reset_index().transpose().iloc[:, [1,2,0,4]]

,1,2,0,4
model,linear,logistic,ising,non_linear
accuracy,0.83(0.17),0.83(0.05),0.94(0.04),0.74(0.09)
time,14.59(5.6),69.45(53.47),32.26(17.88),534.49(337.72)


In [31]:
df = pd.read_csv("skscope_experiment_jit.csv")
df[df.jit == True].groupby(['method', 'model'])['accuracy'].apply(
    lambda x: str(x.mean().round(2)) + "(" + str(x.std().round(2)) + ")"
).unstack().reset_index().iloc[[4,3,2,1,0,5], [0,2,3,1,5]]

model,method,linear,logistic,ising,non_linear
4,OMP,1.0(0.01),0.91(0.05),0.98(0.03),0.77(0.09)
3,IHT,0.79(0.04),0.97(0.03),0.95(0.05),0.78(0.09)
2,HTP,1.0(0.0),0.84(0.05),0.97(0.03),0.78(0.09)
1,GraSP,1.0(0.0),0.87(0.07),0.99(0.01),0.78(0.08)
0,FoBa,1.0(0.0),0.92(0.06),1.0(0.01),0.77(0.09)
5,SCOPE,1.0(0.0),0.94(0.04),1.0(0.01),0.77(0.09)


In [33]:
df = pd.read_csv("skscope_experiment_trend_filter.csv")
print(df.groupby(['method', 'model'])[["accuracy", "time"]].agg(
    lambda x: str(x.mean().round(2)) + "(" + str(x.std().round(2)) + ")"
).unstack().reset_index().iloc[[5,3,2,1,0,6,4]])

      method     accuracy         time
model        trend_filter trend_filter
5        OMP   0.86(0.03)   1.77(0.57)
3        IHT    0.08(0.0)   0.76(0.28)
2        HTP   0.47(0.03)   0.71(0.23)
1      GraSP   0.78(0.12)   0.95(0.38)
0       FoBa     1.0(0.0)   8.27(1.66)
6      SCOPE   0.98(0.02)   4.73(1.13)
4      LASSO   0.21(0.04)  19.75(9.57)


In [42]:
df = pd.read_csv("gurobi_experiment-1.csv")
print(df.groupby(['model'])[["accuracy", "time"]].agg(
    lambda x: str(x.mean().round(2)) + "(" + str(x.std().round(2)) + ")"
).stack().reset_index().transpose())

                0              1           2              3
model      linear         linear  non_linear     non_linear
level_1  accuracy           time    accuracy           time
0        1.0(0.0)  1009.94(0.66)  0.79(0.08)  1003.88(1.53)


In [43]:
df = pd.read_csv("gurobi_experiment-trend_filter.csv")
print(df.groupby(['model'])[["accuracy", "time"]].agg(
    lambda x: str(x.mean().round(2)) + "(" + str(x.std().round(2)) + ")"
).stack().reset_index().transpose())

                    0              1
model    trend_filter   trend_filter
level_1      accuracy           time
0            1.0(0.0)  1013.16(0.62)


In [2]:
df = pd.read_csv("skscope_experiment_jit.csv")

df = df.pivot(
    index=['model', 'seed', 'method'],
    columns='jit',
    values='time',
).reset_index()

df['acceleration'] = df[False] / df[True]
print(df.groupby(['method', 'model'])['acceleration'].mean().unstack().round(2).reset_index().to_markdown())

|    | method   |   ising |   linear |   logistic |   multitask |   non_linear |   trend_filter |
|---:|:---------|--------:|---------:|-----------:|------------:|-------------:|---------------:|
|  0 | FoBa     |   14.76 |    11.93 |      19.16 |        7.02 |         4.32 |           2.97 |
|  1 | GraSP    |   10.34 |     5.76 |      31.63 |        6.73 |         1.07 |           1.81 |
|  2 | HTP      |   13.26 |     5.34 |      13.55 |       11.16 |         1.21 |           0.89 |
|  3 | IHT      |    4.89 |     1.06 |       3.28 |        1.84 |         0.53 |           0.25 |
|  4 | OMP      |   16.16 |    11.33 |      20.88 |        9.82 |         2.83 |           0.9  |
|  5 | SCOPE    |   11.21 |     5.24 |      17.26 |        2.06 |         2.01 |           3.21 |


|          |   Ising model |   Linear regression |   Logistic regression |   Multi-task learning |   Nonlinear feature selection |   Trend filtering |
|:---------|--------:|---------:|-----------:|------------:|-------------:|---------------:|
| FobaSolver     |   14.76 |    11.93 |      19.16 |        7.02 |         4.32 |           2.97 |
| GraspSolver    |   10.34 |     5.76 |      31.63 |        6.73 |         1.07 |           1.81 |
| HTPSolver      |   13.26 |     5.34 |      13.55 |       11.16 |         1.21 |           0.89 |
| IHTSolver      |    4.89 |     1.06 |       3.28 |        1.84 |         0.53 |           0.25 |
| OMPSolver      |   16.16 |    11.33 |      20.88 |        9.82 |         2.83 |           0.9  |
| ScopeSolver    |   11.21 |     5.24 |      17.26 |        2.06 |         2.01 |           3.21 |

\begin{table}[htbp]
  \vspace*{-0.35cm}
  \begin{center}
    {\small
    \begin{tabular}{c|cc|cc|cc}
      \toprule
      Method & \multicolumn{2}{c|}{Linear regression}     & \multicolumn{2}{c|}{Logistic regression} & \multicolumn{2}{c}{Trend filtering} \\ \cline{2-7} 
      & JIT time           & non-JIT time            & JIT time          & non-JIT time           & JIT time           & non-JIT time             \\ \hline
      OMPSolver               &  2.45(0.68) &    26.74(5.99) &  1.66(0.67) &    32.78(10.32) &    1.6(0.65) &  1.43(0.68) \\
      IHTSolver               &  3.42(0.88) &     3.54(1.11) &  1.06(0.67) &      3.59(3.77) &    2.8(1.35) &  0.68(0.35) \\
      HTPSolver               &  4.14(1.25) &     21.7(8.55) &  2.37(0.92) &    32.13(17.28) &   1.71(0.85) &  1.46(0.82) \\
      GraspSolver             &  1.16(0.38) &     6.57(2.35) &   12.7(8.2) &  408.14(246.79) &   1.74(0.74) &  3.14(1.95) \\
      FobaSolver              &   11.7(2.9) &  132.04(17.53) &  6.31(2.15) &   116.23(33.13) &   1.89(0.84) &  5.46(2.24) \\
      ScopeSolver             &   2.11(0.7) &    10.73(3.36) &  3.24(2.67) &    53.88(38.12) &   2.85(1.29) &  8.93(3.54) \\
      \midrule
      Method & \multicolumn{2}{c|}{Multi-task learning} & \multicolumn{2}{c|}{Ising model}    & \multicolumn{2}{c}{Nonlinear feature selection}   \\ \cline{2-7} 
      & JIT time           & non-JIT time            & JIT time          & non-JIT time           & JIT time           & non-JIT time             \\ \hline
      OMPSolver               &  4.07(0.48) &     39.8(5.22) &   2.86(0.86) &   45.64(14.47) &  11.53(3.61) &   32.61(10.47) \\
      IHTSolver               &  1.24(0.23) &     2.26(0.49) &   3.24(1.43) &    15.23(5.69) &   6.37(2.32) &     3.36(2.06) \\
      HTPSolver               &  21.5(4.56) &  235.89(64.94) &   5.26(2.03) &   70.71(34.32) &  10.82(7.86) &    12.61(8.33) \\
      GraspSolver             &  0.87(0.31) &     5.95(2.75) &   1.02(0.44) &    10.62(5.77) &   7.34(2.75) &     7.65(3.05) \\
      FobaSolver              &  6.65(0.65) &    46.59(6.96) &  11.59(3.55) &  165.58(42.08) &   31.26(8.8) &  132.27(32.48) \\
      ScopeSolver             &  7.28(4.84) &    14.02(9.26) &   1.69(0.67) &    18.65(7.79) &     8.6(2.7) &    17.03(5.52) \\
      \bottomrule
    \end{tabular}
    }
  \end{center}
\end{table}